In [49]:
import pandas as pd
import difflib
import fiona

In [51]:
#Lê dados
crimes = pd.read_csv('raw_data.csv', sep=';', encoding="ISO-8859-1")

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (26,27,29,30,32,33,35,36,38,39,41,42,44,45,47,48,50,51,53,54,56,57,59,60,62,63,65,66,68,69,71,72,74,75,77,78,80,81,83,84,86,87,89,90,92,93,95,96,98,99,101,102,104,105,107,108,110,111,113,114,116,117) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [52]:
#Drop colunas desnecessárias
crimes = crimes.drop(crimes.columns[10:], axis=1)
#Drop cidades que não são Porto Alegre
crimes =  crimes[crimes['Municipio Fato'] == 'PORTO ALEGRE']
#Drop NA na coluna bairros. Converte coluna para lower
crimes =  crimes[crimes['Bairro'].notna()]
#Lower case
crimes['Bairro'] = crimes['Bairro'].apply(lambda x: x.lower())
#Tira acentos
crimes['Bairro'] = crimes['Bairro'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
#Arruma alguns erros comuns de gramática
crimes['Bairro'] = crimes['Bairro'].str.replace('vl', 'vila')
crimes['Bairro'] = crimes['Bairro'].str.replace('sta', 'santa')
#Arruma alguns bairros errados, segundo o mapa utilizado para a análise
crimes['Bairro'] = crimes['Bairro'].str.replace('protasio alves', 'morro santana')
crimes.loc[crimes['Bairro'] == 'centro', 'Bairro'] = 'centro historico'
#Upper case
crimes['Bairro'] = crimes['Bairro'].apply(lambda x: x.upper())


In [53]:
porto_alegre = fiona.open("shapesbairros2016/Bairros_2016.shp")

bairros = list()

for i in porto_alegre:
    bairros.append(i['properties']['NOME'])

In [54]:
crimes['Bairro'] = crimes['Bairro'].apply(lambda x: difflib.get_close_matches(x, bairros, n=1))

In [73]:
crimes = crimes[crimes["Bairro"].str.len() != 0]

In [74]:
crimes["Bairro"] = crimes["Bairro"].apply(lambda x: x[0])

<ipython-input-74-9098f99b7a14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crimes["Bairro"] = crimes["Bairro"].apply(lambda x: x[0])


In [75]:
crimes.to_csv('processed_data.csv')